# E-commerce Data Exploratory Analysis

This notebook focuses on exploring the cleaned e-commerce dataset through:
1. Sales trends analysis
2. Customer behavior analysis
3. Product performance analysis
4. Time-based patterns

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sys
sys.path.append('..')

from src.visualization import DataVisualizer
from src.feature_engineering import FeatureEngineer
from src.config import PROCESSED_DATA_DIR

%matplotlib inline
plt.style.use('seaborn')

In [ ]:
# Load cleaned data
df = pd.read_csv(PROCESSED_DATA_DIR / 'cleaned_sales_data.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

# Initialize visualizer
visualizer = DataVisualizer(df)

## 1. Sales Trends Analysis

In [ ]:
# Daily sales trend
visualizer.plot_sales_trend(time_unit='D')

# Monthly sales trend
visualizer.plot_sales_trend(time_unit='M')

# Calculate key metrics
daily_sales = df.groupby('order_date')['price'].sum()
print(f"Average daily sales: ${daily_sales.mean():.2f}")
print(f"Sales growth: {((daily_sales.iloc[-1] / daily_sales.iloc[0] - 1) * 100):.2f}%")

## 2. Customer Behavior Analysis

In [ ]:
# Feature engineering for customer analysis
engineer = FeatureEngineer(df)
df_with_features = engineer.create_customer_features()

# Customer purchase frequency distribution
plt.figure(figsize=(12, 6))
sns.histplot(data=df_with_features, x='total_orders', bins=30)
plt.title('Distribution of Customer Purchase Frequency')
plt.show()

# Customer lifetime value analysis
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df_with_features, x='total_spent', y='purchase_frequency')
plt.title('Customer Lifetime Value Analysis')
plt.show()

## 3. Product Performance Analysis

In [ ]:
# Add product features
df_with_features = engineer.create_product_features()

# Top performing products
top_products = df_with_features.groupby('product_id').agg({
    'price': ['sum', 'count'],
    'product_avg_price': 'first'
}).sort_values(('price', 'sum'), ascending=False).head(10)

# Visualize product performance
plt.figure(figsize=(15, 6))
sns.barplot(data=top_products.reset_index(), x='product_id', y=('price', 'sum'))
plt.title('Top 10 Products by Revenue')
plt.xticks(rotation=45)
plt.show()

## 4. Time-based Patterns

In [ ]:
# Add time features
df_with_features = engineer.create_time_features()

# Daily patterns
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_with_features, x='order_dayofweek', y='price')
plt.title('Sales Distribution by Day of Week')
plt.show()

# Monthly patterns
plt.figure(figsize=(12, 6))
monthly_sales = df_with_features.groupby('order_month')['price'].sum()
plt.plot(monthly_sales.index, monthly_sales.values, marker='o')
plt.title('Monthly Sales Pattern')
plt.show()

## 5. Correlation Analysis

In [ ]:
# Select numerical columns for correlation analysis
numerical_cols = df_with_features.select_dtypes(include=['float64', 'int64']).columns

# Create correlation matrix
correlation_matrix = df_with_features[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Identify strongest correlations
strong_correlations = pd.DataFrame()
for col in correlation_matrix.columns:
    strong_corr = correlation_matrix[col][abs(correlation_matrix[col]) > 0.5]
    strong_corr = strong_corr[strong_corr.index != col]
    if not strong_corr.empty:
        print(f"\nStrong correlations with {col}:")
        print(strong_corr)

## 6. Key Insights Summary

In [ ]:
# Calculate and print key business metrics
insights = {
    'Total Revenue': f"${df_with_features['price'].sum():,.2f}",
    'Average Order Value': f"${df_with_features['price'].mean():,.2f}",
    'Total Customers': df_with_features['customer_id'].nunique(),
    'Total Products': df_with_features['product_id'].nunique(),
    'Best Selling Category': df_with_features.groupby('category')['price'].sum().idxmax(),
    'Peak Sales Month': df_with_features.groupby('order_month')['price'].sum().idxmax()
}

for metric, value in insights.items():
    print(f"{metric}: {value}")